In [4]:
import os
import random
import shutil
import cv2
from ultralytics import YOLO
import torch
from glob import glob
from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split

# ===============================
# USER CONFIG
# ===============================
dataset_root = "synthetic_braille_dataset"  # <-- change to your folder with 47 subfolders
output_root = "braille_detect_dataset"
train_ratio = 0.8
# ===============================

# Prepare output folders
for sub in ["images/train", "images/val", "labels/train", "labels/val"]:
    os.makedirs(os.path.join(output_root, sub), exist_ok=True)

# Collect all images
class_dirs = [d for d in os.listdir(dataset_root) if os.path.isdir(os.path.join(dataset_root, d))]
print(f"Found {len(class_dirs)} classes")

# Assign numeric class IDs
class_to_id = {cls_name: i for i, cls_name in enumerate(sorted(class_dirs))}

# Build dataset list
data = []
for cls_name in class_dirs:
    img_paths = glob(os.path.join(dataset_root, cls_name, "*.*"))
    for img in img_paths:
        if img.lower().endswith(('.jpg', '.jpeg', '.png')):
            data.append((img, cls_name))

print(f"Total images found: {len(data)}")

# Split train/val
train_data, val_data = train_test_split(data, train_size=train_ratio, random_state=42, shuffle=True)

# Helper to create YOLO labels
def make_label_file(img_path, label_path, class_id):
    img = cv2.imread(img_path)
    if img is None:
        return
    h, w, _ = img.shape
    # full image box (normalized)
    x_center, y_center, box_w, box_h = 0.5, 0.5, 1.0, 1.0
    with open(label_path, 'w') as f:
        f.write(f"{class_id} {x_center} {y_center} {box_w} {box_h}\n")

# Copy images and create labels
def process_split(data_split, split_name):
    for img_path, cls_name in tqdm(data_split, desc=f"Preparing {split_name}"):
        class_id = class_to_id[cls_name]
        img_name = os.path.basename(img_path)
        dst_img = os.path.join(output_root, "images", split_name, img_name)
        dst_lbl = os.path.join(output_root, "labels", split_name, os.path.splitext(img_name)[0] + ".txt")
        shutil.copy(img_path, dst_img)
        make_label_file(dst_img, dst_lbl, class_id)

process_split(train_data, "train")
process_split(val_data, "val")

# Create YAML
yaml_path = os.path.join(output_root, "data.yaml")
with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(f"path: {os.path.abspath(output_root)}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write(f"nc: {len(class_dirs)}\n")
    f.write(f"names: {list(sorted(class_dirs))}\n")

print("\n✅ Dataset prepared successfully!")

# ===============================
# Train YOLO-Nano (yolov8n)
# ===============================
print("\n🚀 Starting YOLO-Nano training...")
model = YOLO('yolov8n.pt')

model.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,
    name='braille_nano',
    device=0 if torch.cuda.is_available() else 'cpu'
)

print("\n✅ Training complete! Check 'runs/detect/braille_nano' for results.")


Found 47 classes
Total images found: 11750


Preparing val: 100%|██████████| 2350/2350 [00:18<00:00, 127.36it/s]



✅ Dataset prepared successfully!

🚀 Starting YOLO-Nano training...
Ultralytics 8.3.228  Python-3.10.0 torch-2.9.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=braille_detect_dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=braille_nano2, nbs=64, nms=False, opset=None, optimize

KeyboardInterrupt: 